In [ ]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

### Read AWS Credentials

In [ ]:
# Define the path to the Delta table
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# Read the Delta table to a Spark DataFrame
aws_keys_df = spark.read.format("delta").load(delta_table_path)

In [ ]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [ ]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0a5040edb649-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/user-0a5040edb649-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
try:
    dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)
    print(f"Successfully mounted {SOURCE_URL} to {MOUNT_NAME}.")
except Exception as e:
    if "already mounted" in str(e):
        print(f"{MOUNT_NAME} already mounted. You can unmount using dbutils.fs.unmount('{MOUNT_NAME}') if needed.")
    else:
        print(f"Error mounting {SOURCE_URL} to {MOUNT_NAME}: {e}")


/mnt/user-0a5040edb649-bucket already mounted. You can unmount using dbutils.fs.unmount('/mnt/user-0a5040edb649-bucket') if needed.

### Read data from S3 bucket into dataframe

In [ ]:
dbutils.fs.ls("mnt/user-0a5040edb649-bucket/topics")

Out[362]: [FileInfo(path='dbfs:/mnt/user-0a5040edb649-bucket/topics/0a5040edb649.geo/', name='0a5040edb649.geo/', size=0, modificationTime=1711585975218),
 FileInfo(path='dbfs:/mnt/user-0a5040edb649-bucket/topics/0a5040edb649.pin/', name='0a5040edb649.pin/', size=0, modificationTime=1711585975218),
 FileInfo(path='dbfs:/mnt/user-0a5040edb649-bucket/topics/0a5040edb649.user/', name='0a5040edb649.user/', size=0, modificationTime=1711585975218)]

In [ ]:
%sql
-- Disable format checks during the reading of Delta tables 
SET spark.databricks.delta.formatCheck.enabled=false

key,value
spark.databricks.delta.formatCheck.enabled,false


In [ ]:
topics = [".pin", ".geo", ".user"]

def load_topics_into_dataframe(topic):
    # File path for topic files
    file_path = f"/mnt/user-0a5040edb649-bucket/topics/0a5040edb649{topic}/partition=0/*.json"
    file_type = "json"
    
    # Ask Spark to infer the schema
    infer_schema = "true"
    
    # Read JSONs from mounted S3 bucket into Spark DataFrame
    df = spark.read.format(file_type) \
        .option("inferSchema", infer_schema) \
        .load(file_path)
    return df

# Iterate over topics and create DataFrames for each topic
for suffix in topics:
    # Create DataFrame for each topic
    create_df = f"df_{suffix[1:]} = load_topics_into_dataframe('{suffix}')"
    exec(create_df)
    # Show DataFrame for each topic
    show_df = f"df_{suffix[1:]}.show()"
    exec(show_df)

+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 category| description|downloaded|follower_count| image_src|index|is_image_or_video| poster_name| save_location| tag_list| title| unique_id|
+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 christmas|Christmas decorat...| 1| 46k|https://i.pinimg....| 2482| video|Life on Summerhil...|Local save in /da...|Diy Christmas Dec...|FORNT PORCH CHRIS...|08604f20-fa17-4b9...|
 travel|This Costa Rica i...| 1| 10k|https://i.pinimg....|10138| image|Wanderlust Chloe ...|Local save in /da...|Costa Rica Travel...|14 Amazing Things...|927c4658-cc3f-4b9...|
 travel|This Costa Rica i...| 1| 10k|https://i.pinimg....|10138| image|Wanderlust Chloe ...|Local save in /da...|Costa Rica Travel...|14 Amazing Things...|927c4658-cc3f-4b9...|
 christmas|My favorite 75+ N...| 1| 31k|https://i.pinimg....| 2604| image| Everyday Wholesome|Local save in /da...|Colorful Christma...|75+ Neutral Chris...|087b0fa9-f901-426...|
diy-and-crafts|This post may con...| 1| 892k|https://i.pinimg....| 3156| image|Michelle {CraftyM...|Local save in /da...|Christmas Gifts F...|Handprint Reindee...|fa6e31a4-18c2-4ec...|
diy-and-crafts|This post may con...| 1| 892k|https://i.pinimg....| 3156| image|Michelle {CraftyM...|Local save in /da...|Christmas Gifts F...|Handprint Reindee...|fa6e31a4-18c2-4ec...|
 travel|See families trav...| 1| 9k|https://i.pinimg....|10119| image|OUR NEXT ADVENTUR...|Local save in /da...|Family Vacation D...|How to Afford Fam...|40eab9ba-7812-4f2...|
diy-and-crafts|Easy Christmas tr...| 1| 3k|https://i.pinimg....| 3419| image|Kids Crafts & Fre...|Local save in /da...|Christmas Crafts ...|Easy Christmas Tr...|d0b80187-0171-49b...|
 tattoos|"Minimalist Tatto...| 1| 54k|https://i.pinimg....| 9093| image|Beautyholo | Late...|Local save in /da...|Small Tattoos Men...|51 Unique Minimal...|8e478adb-0e3f-404...|
 finance|If you love budge...| 1| 26k|https://i.pinimg....| 5494| image|Living Low Key | ...|Local save in /da...|Financial Peace,F...|Dave Ramsey's 7 B...|8fb2af68-543b-463...|
 finance|If you love budge...| 1| 26k|https://i.pinimg....| 5494| image|Living Low Key | ...|Local save in /da...|Financial Peace,F...|Dave Ramsey's 7 B...|8fb2af68-543b-463...|
 finance|If you love budge...| 1| 26k|https://i.pinimg....| 5494| image|Living Low Key | ...|Local save in /da...|Financial Peace,F...|Dave Ramsey's 7 B...|8fb2af68-543b-463...|
diy-and-crafts|Celebrate warmer ...| 1| 108k|https://i.pinimg....| 2858| image|This Tiny Blue H...|Local save in /da...|Spring Crafts For...|50 Popular Spring...|26f81da4-26a9-465...|
 christmas|Here are the best...| 1| 500k|https://i.pinimg....| 2418| image| HikenDip|Local save in /da...|Farmhouse Christm...|100 DIY Christmas...|da8745a6-5160-46c...|
 christmas|Here are the best...| 1| 500k|https://i.pinimg....| 2418| image| HikenDip|Local save in /da...|Farmhouse Christm...|100 DIY Christmas...|da8745a6-5160-46c...|
 quotes|Trying to create ...| 1| 42k|https://i.pinimg....| 8312| image|TheFab20s | Trave...|Local save in /da...|Positive Self Aff...|8 Vision Board Id...|ca3c9bb0-7281-4b9...|
 quotes|summcoco gives yo...| 1| 306k|https://i.pinimg....| 7922| image|Sumcoco | Decor I...|Local save in /da...|Life Quotes Love,...|45 Top Life Quote...|a584581c-1b38-473...|
 quotes|summcoco gives yo...| 1| 306k|https://i.pinimg....| 7922| image|Sumcoco | Decor I...|Local save in /da...|Life Quotes Love,...|45 Top Life Quote...|a584581c-1b38-473...|
 art|Use your mini wor...| 1| 4k|https://i.pinimg....| 771| image|Taming Little Mon...|Local save in /da...|African Art Proje...|African Sunset Sh...|a5021766-a8aa-4dc...|
event-planning|Personalize your ...| 1| 111|http

### Clean Pinterest posts datafram


In [ ]:
# Replace empty entries and entries with no relevant data in each column with Nones
missing_values = { 
    "follower_count": "User Info Error",
    "description": "No description available%",
    "tag_list": "N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",
    "image_src": "Image src error.",
    "poster_name": "User Info Error",
    "title": "No Title Data Available"
}

for column, value in missing_values.items():
    df_pin = df_pin.withColumn(column, when(col(column).like(value), None).otherwise(col(column)))

# Perform transformations on the follower_count to ensure every entry is a number
df_pin = df_pin.withColumn("follower_count", regexp_replace("follower_count", "k", "000"))
df_pin = df_pin.withColumn("follower_count", regexp_replace("follower_count", "M", "000000"))
# Cast follower_count column to integer type
df_pin = df_pin.withColumn("follower_count", col("follower_count").cast('int'))
# Clean the save_location column to include only the save location path
df_pin = df_pin.withColumn("save_location", regexp_replace("save_location", "Local save in ", ""))
# Rename the index column to ind
df_pin = df_pin.withColumnRenamed("index", "ind")
# Rearrange columns
pin_column_order = [
    "ind",
    "unique_id",
    "title",
    "description",
    "follower_count",
    "poster_name",
    "tag_list",
    "is_image_or_video",
    "image_src",
    "save_location",
    "category"
]
df_pin = df_pin.select(new_pin_column_order)
df_pin.show()

+-----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
 ind| unique_id| title| description|follower_count| poster_name| tag_list|is_image_or_video| image_src| save_location| category|
+-----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
 2482|08604f20-fa17-4b9...|FORNT PORCH CHRIS...|Christmas decorat...| 46000|Life on Summerhil...|Diy Christmas Dec...| video|https://i.pinimg....| /data/christmas| christmas|
10138|927c4658-cc3f-4b9...|14 Amazing Things...|This Costa Rica i...| 10000|Wanderlust Chloe ...|Costa Rica Travel...| image|https://i.pinimg....| /data/travel| travel|
10138|927c4658-cc3f-4b9...|14 Amazing Things...|This Costa Rica i...| 10000|Wanderlust Chloe ...|Costa Rica Travel...| image|https://i.pinimg....| /data/travel| travel|
 2604|087b0fa9-f901-426...|75+ Neutral Chris...|My favorite 75+ N...| 31000| Everyday Wholesome|Colorful Christma...| image|https://i.pinimg....| /data/christmas| christmas|
 3156|fa6e31a4-18c2-4ec...|Handprint Reindee...|This post may con...| 892000|Michelle {CraftyM...|Christmas Gifts F...| image|https://i.pinimg....|/data/diy-and-crafts|diy-and-crafts|
 3156|fa6e31a4-18c2-4ec...|Handprint Reindee...|This post may con...| 892000|Michelle {CraftyM...|Christmas Gifts F...| image|https://i.pinimg....|/data/diy-and-crafts|diy-and-crafts|
10119|40eab9ba-7812-4f2...|How to Afford Fam...|See families trav...| 9000|OUR NEXT ADVENTUR...|Family Vacation D...| image|https://i.pinimg....| /data/travel| travel|
 3419|d0b80187-0171-49b...|Easy Christmas Tr...|Easy Christmas tr...| 3000|Kids Crafts & Fre...|Christmas Crafts ...| image|https://i.pinimg....|/data/diy-and-crafts|diy-and-crafts|
 9093|8e478adb-0e3f-404...|51 Unique Minimal...|"Minimalist Tatto...| 54000|Beautyholo | Late...|Small Tattoos Men...| image|https://i.pinimg....| /data/tattoos| tattoos|
 5494|8fb2af68-543b-463...|Dave Ramsey's 7 B...|If you love budge...| 26000|Living Low Key | ...|Financial Peace,F...| image|https://i.pinimg....| /data/finance| finance|
 5494|8fb2af68-543b-463...|Dave Ramsey's 7 B...|If you love budge...| 26000|Living Low Key | ...|Financial Peace,F...| image|https://i.pinimg....| /data/finance| finance|
 5494|8fb2af68-543b-463...|Dave Ramsey's 7 B...|If you love budge...| 26000|Living Low Key | ...|Financial Peace,F...| image|https://i.pinimg....| /data/finance| finance|
 2858|26f81da4-26a9-465...|50 Popular Spring...|Celebrate warmer ...| 108000|This Tiny Blue H...|Spring Crafts For...| image|https://i.pinimg....|/data/diy-and-crafts|diy-and-crafts|
 2418|da8745a6-5160-46c...|100 DIY Christmas...|Here are the best...| 500000| HikenDip|Farmhouse Christm...| image|https://i.pinimg....| /data/christmas| christmas|
 2418|da8745a6-5160-46c...|100 DIY Christmas...|Here are the best...| 500000| HikenDip|Farmhouse Christm...| image|https://i.pinimg....| /data/christmas| christmas|
 8312|ca3c9bb0-7281-4b9...|8 Vision Board Id...|Trying to create ...| 42000|TheFab20s | Trave...|Positive Self Aff...| image|https://i.pinimg....| /data/quotes| quotes|
 7922|a584581c-1b38-473...|45 Top Life Quote...|summcoco gives yo...| 306000|Sumcoco | Decor I...|Life Quotes Love,...| image|https://i.pinimg....| /data/quotes| quotes|
 7922|a584581c-1b38-473...|45 Top Life Quote...|summcoco gives yo...| 306000|Sumcoco | Decor I...|Life Quotes Love,...| image|https://i.pinimg....| /data/quotes| quotes|
 771|a5021766-a8aa-4dc...|African Sunset Sh...|Use your mini wor...| 4000|Taming Little Mon...|African Art Proje...| image|https://i.pinimg....| /data/art| art|
 4508|9064f4a2-2753-476...|Custom Event and ...|Personalize your ...| 111|Life of Neon | Cu...|Our Wedding,Weddi...| image|https://i.pinimg....|/data/event-planning|event-planning|
+-----+-----

### Clean Geolocation dataframe

In [ ]:
# Create a new column coordinates that contains an array based on the latitude and longitude columnse
df_geo = df_geo.withColumn("coordinates", array(col("latitude"), col("longitude")))
# Drop the latitude and longitude columns
df_geo = df_geo.drop("latitude", "longitude")
# Convert timestamp column from a string to a timestamp type
df_geo = df_geo.withColumn("timestamp", to_timestamp("timestamp"))
# Rearrange columns
geo_column_order = [
    "ind",
    "country",
    "coordinates",
    "timestamp"
]
df_geo = df_geo.select(geo_column_order)
df_geo.show()

+-----+--------------------+--------------------+-------------------+
 ind| country| coordinates| timestamp|
+-----+--------------------+--------------------+-------------------+
 2418|Antarctica (the t...|[-88.4642, -171.061]|2022-05-27 11:30:59|
10794|Cocos (Keeling) I...|[-89.5236, -154.567]|2022-01-01 02:26:50|
 2074|Central African R...| [-52.3213, -50.11]|2019-11-03 05:41:59|
 7922| Antigua and Barbuda|[-88.0974, -172.052]|2021-01-27 09:14:19|
 9979| Dominican Republic| [14.9967, -120.682]|2018-07-18 19:01:46|
 2923| Cote d'Ivoire|[-84.6302, -164.507]|2019-09-08 22:53:09|
 8304| French Guiana| [-28.8852, -164.87]|2019-09-13 04:50:29|
 4315| Cote d'Ivoire| [-45.8508, 66.1003]|2019-12-15 03:51:28|
 5069| Azerbaijan|[-63.0063, -157.474]|2021-03-20 09:32:44|
 6145| Mozambique|[-65.9079, -143.845]|2019-12-05 02:09:44|
 428| Bangladesh|[-83.4105, -150.788]|2020-09-27 18:46:41|
 7343| Australia|[-65.4428, -161.684]|2018-08-06 12:17:07|
 8653| Seychelles| [48.4569, -139.658]|2022-04-11 18:30:19|
 5494| Bulgaria|[-82.6768, -129.202]|2021-07-21 02:02:35|
 6063| Anguilla|[-89.1797, -174.015]|2021-07-20 09:02:47|
 3454| Cambodia|[-0.375174, 49.8106]|2021-07-25 02:20:29|
10625| Jamaica|[-84.4944, -81.0613]|2018-07-13 11:51:15|
 9875| Barbados|[-74.3382, -110.484]|2020-03-20 13:03:18|
 8887| Botswana|[-28.0137, -160.708]|2021-09-19 05:27:43|
 7528| Albania|[-89.9787, -173.293]|2020-08-28 03:52:47|
+-----+--------------------+--------------------+-------------------+
only showing top 20 rows

### Clean Users dataframe

In [ ]:
# Create a user_name from the first_name and last_name columns
df_user = df_user.withColumn("user_name", concat_ws(" ", "first_name", "last_name"))
# Drop the first_name and last_name columns
df_user = df_user.drop("first_name", "last_name")
# Convert date_joined column from a string to a timestamp type
df_user = df_user.withColumn("date_joined", to_timestamp("date_joined"))
# Rearrange columns
user_column_order = [
    "ind",
    "user_name",
    "age",
    "date_joined"
]
df_user = df_user.select(user_column_order)
df_user.show()

+-----+----------------+---+-------------------+
 ind| user_name|age| date_joined|
+-----+----------------+---+-------------------+
 8731|Andrea Alexander| 21|2015-11-10 09:27:42|
 8887|Austin Rodriguez| 24|2016-03-31 20:56:39|
 4315| Michelle Prince| 36|2015-12-20 16:38:13|
10625| Christian Lang| 32|2017-10-10 20:09:33|
 9672| Jennifer Hudson| 22|2016-02-11 20:46:04|
 1313| Brittany Jones| 32|2016-04-02 03:51:23|
10794| Thomas Turner| 34|2016-12-22 00:02:02|
 2959| David Griffith| 20|2016-01-07 19:49:22|
 9875| Brendan Joseph| 26|2015-12-20 10:28:00|
 2074| Annette Forbes| 21|2016-01-03 15:42:12|
 1014| Gregory Ramsey| 56|2016-10-15 17:21:39|
 8304| Charles Berry| 25|2015-12-28 04:21:39|
 6063| Corey Andrews| 23|2015-11-25 13:36:22|
 3454| Robert Murphy| 48|2017-09-26 16:31:56|
 7554| Cheryl Huerta| 20|2017-04-11 16:35:33|
 6145| Emily Hawkins| 30|2015-12-25 20:24:37|
 9979| Kaylee Miller| 31|2016-11-09 19:50:51|
 4913| Angela Brooks| 24|2015-11-20 05:37:11|
 2863| Dylan Holmes| 32|2016-10-23 14:06:51|
 5730| Rachel Davis| 36|2015-12-08 20:02:43|
+-----+----------------+---+-------------------+
only showing top 20 rows

In [ ]:
from pyspark.sql.window import Window
# Join df_pin and df_geo dataframes on index coumn
df_pin_geo = df_pin.join(df_geo, df_pin.ind == df_geo.ind)

### Find the most popular category people post to based on their country.

In [ ]:

# Define a window function to partition by country and order by category_count in descending order.
windowCategoryPerCountry = Window.partitionBy("country").orderBy(col("category_count").desc())

# Group the data by "country" and "category", and count the number of posts in each category.
# Add a column "category_count" to store the count.
# Add a rank column using row_number() function over the defined window function.
# Filter the rows where rank is 1 to get the category with the highest post count for each country.
# Drop the rank column and display the result.
df_pin_geo.groupBy("country", "category") \
    .agg(count("category").alias("category_count")) \
    .withColumn("rank", row_number().over(windowCategoryPerCountry)) \
    .filter(col("rank") == 1) \
    .drop("rank") \
    .show()

+--------------------+--------------+--------------+
 country| category|category_count|
+--------------------+--------------+--------------+
 Albania| mens-fashion| 10|
 Angola|diy-and-crafts| 2|
 Anguilla| home-decor| 3|
Antarctica (the t...| christmas| 2|
 Antigua and Barbuda| quotes| 2|
 Armenia|diy-and-crafts| 10|
 Aruba| tattoos| 3|
 Australia| mens-fashion| 2|
 Austria| travel| 2|
 Azerbaijan|event-planning| 3|
 Bangladesh| art| 2|
 Barbados| travel| 2|
 Botswana| tattoos| 2|
 Bulgaria| finance| 3|
 Cambodia|diy-and-crafts| 3|
Central African R...| christmas| 2|
Cocos (Keeling) I...| vehicles| 3|
 Colombia| finance| 8|
 Cote d'Ivoire|diy-and-crafts| 3|
 Dominican Republic| travel| 2|
+--------------------+--------------+--------------+
only showing top 20 rows

### Find how many posts each category had between 2018 and 2022

In [ ]:
# Define a window function to partition by post_year and order by category_count in descending order.
windowCategoryPerYear = Window.partitionBy("post_year").orderBy(col("category_count").desc())

# Add a new column "post_year" to df_pin_geo by extracting the year from the "timestamp" column.
# Group the data by "post_year" and "category", and count the number of posts in each category.
# Add a rank column using row_number() function over the defined window function.
# Filter the rows where rank is 1 to get the category with the highest post count in each year.
# Drop the rank column and display the result.
df_pin_geo.withColumn("post_year", year("timestamp")) \
    .groupBy("post_year", "category") \
    .agg(count("category").alias("category_count")) \
    .withColumn("rank", row_number().over(windowCategoryPerYear)) \
    .filter(col("rank") == 1) \
    .drop("rank") \
    .show()

+---------+--------------+--------------+
post_year| category|category_count|
+---------+--------------+--------------+
 2018|diy-and-crafts| 5|
 2019|diy-and-crafts| 5|
 2020| mens-fashion| 10|
 2021| finance| 11|
 2022| vehicles| 3|
+---------+--------------+--------------+

### For each country, find the user with the most followers.
### Then, find the country with the user with the most followers.

In [ ]:

# Define a window function to partition by country and order by follower_count in descending order.
windowFollowersPerCountry = Window.partitionBy("country").orderBy(col("follower_count").desc())

# Select the columns "country", "poster_name", "follower_count" from df_pin_geo.
# Add a rank column using row_number() function over the defined window function.
# Filter the rows where rank is 1 to get the user with the most followers for each country.
# Drop the rank column and rows with null values.
most_followers_per_country = df_pin_geo \
    .select("country", "poster_name", "follower_count") \
    .withColumn("rank", row_number().over(windowFollowersPerCountry)) \
    .filter(col("rank") == 1) \
    .drop("rank") \
    .dropna()

# Order the most_followers_per_country DataFrame by follower_count in descending order
# and select the row with the highest follower_count using limit(1).
most_popular_user = most_followers_per_country \
    .orderBy(col("follower_count").desc()) \
    .limit(1)

# Display the most_popular_user DataFrame.
most_popular_user.show()

+----------+---------------+--------------+
 country| poster_name|follower_count|
+----------+---------------+--------------+
Azerbaijan|Style Me Pretty| 6000000|
+----------+---------------+--------------+

### Find the most popular category people post to per age group

In [ ]:
# Define a window function to partition by age_group and order by category_count in descending order.
windowCategoryPerAge = Window.partitionBy("age_group").orderBy(col("category_count").desc())

# Join df_pin, df_geo, and df_user on the "ind" column to get the required data in df_pin_geo_user.
df_pin_geo_user = df_pin.join(df_geo, on="ind").join(df_user, on="ind")

# Add a new column "age_group" to df_pin_geo_user based on the age range of each user.
# Group the data by "age_group" and "category", and count the number of posts in each category.
# Add a rank column using row_number() function over the defined window function.
# Filter the rows where rank is 1 to get the category with the highest post count in each age group.
# Drop the rank column and display the result.
df_pin_geo_user.withColumn("age_group", when((col("age") >= 18) & (col("age") <= 24), "18-24")
                                   .when((col("age") >= 25) & (col("age") <= 35), "25-35")
                                   .when((col("age") >= 36) & (col("age") <= 50), "36-50")
                                   .when((col("age") > 50), "50+")) \
    .groupBy("age_group", "category") \
    .agg(count("category").alias("category_count")) \
    .withColumn("rank", row_number().over(windowCategoryPerAge)) \
    .filter(col("rank") == 1) \
    .drop("rank") \
    .show()

+---------+--------------+--------------+
age_group| category|category_count|
+---------+--------------+--------------+
 18-24| mens-fashion| 25|
 25-35|diy-and-crafts| 19|
 36-50| finance| 16|
 50+| beauty| 2|
+---------+--------------+--------------+

### Find the most popular category for different age groups

In [ ]:
# Join df_pin and df_user DataFrames on the "ind" column to get the required data in df_pin_user.
df_pin_user = df_pin.join(df_user, on="ind")

# Add a new column "age_group" to df_pin_user based on the age range of each user.
df_pin_user_age = df_pin_user.withColumn("age_group", when((col("age") >= 18) & (col("age") <= 24), "18-24")
                                   .when((col("age") >= 25) & (col("age") <= 35), "25-35")
                                   .when((col("age") >= 36) & (col("age") <= 50), "36-50")
                                   .when((col("age") > 50), "50+"))

# Select the columns "age_group" and "follower_count" from df_pin_user_age DataFrame.
# Group the data by "age_group" and calculate the approximate median follower count using percentile_approx() function.
# Rename the aggregated column to "median_follower_count".
# Order the result DataFrame by "median_follower_count" in descending order and display the result.
df_pin_user_age.select("age_group", "follower_count") \
    .distinct() \
    .groupBy("age_group") \
    .agg(percentile_approx(col("follower_count"), 0.5).alias("median_follower_count")) \
    .orderBy(col("median_follower_count").desc()) \
    .show()

+---------+---------------------+
age_group|median_follower_count|
+---------+---------------------+
 18-24| 211000|
 25-35| 43000|
 36-50| 6000|
 50+| 0|
+---------+---------------------+

### Find the number of users who joined between 2015 and 2020.

In [ ]:
# Join df_pin and df_user DataFrames on "ind" column to get the required data in df_pin_user.
df_pin_user = df_pin.join(df_user, on="ind")

# Add a new column "join_year" to df_pin_user that contains only the year from the "date_joined" column.
df_pin_user_year = df_pin_user.withColumn("join_year", year("date_joined"))

# Select distinct columns "user_name" and "join_year" from df_pin_user_year DataFrame.
# Group the data by "join_year" and calculate the count of join_years using count() function.
# Rename the aggregated column to "number_users_joined".
# Display the result DataFrame.
df_pin_user_year.select("user_name", "join_year") \
    .distinct() \
    .groupBy("join_year") \
    .agg(count(col("join_year")).alias("number_users_joined")) \
    .show()
    
# Print the total number of distinct user_names from df_pin_user_year DataFrame.
print(df_pin_user_year.select("user_name").distinct().count())

+---------+-------------------+
join_year|number_users_joined|
+---------+-------------------+
 2015| 16|
 2016| 12|
 2017| 3|
+---------+-------------------+

31

### Find the median follower count of users based on their joining year

In [ ]:
# Join df_pin and df_user DataFrames on "ind" column to get the required data in df_pin_user.
df_pin_user = df_pin.join(df_user, on="ind")

# Add a new column "join_year" to df_pin_user that contains only the year from the "date_joined" column.
df_pin_user_year = df_pin_user.withColumn("join_year", year("date_joined"))

# Add a new column "age_group" to df_pin_user_year based on the age range of each user.
df_pin_user_year_age = df_pin_user_year.withColumn("age_group", when((col("age") >= 18) & (col("age") <= 24), "18-24")
                                   .when((col("age") >= 25) & (col("age") <= 35), "25-35")
                                   .when((col("age") >= 36) & (col("age") <= 50), "36-50")
                                   .when((col("age") > 50), "50+"))

# Select the columns "user_name, age_group", "join_year" and "follower_count" from df_pin_user_year_age DataFrame.
# Filter the data for join_years between 2015 and 2020.
# Group the data by "join_year" and calculate the approximate median follower count using percentile_approx() function.
# Rename the aggregated column to "median_follower_count".
# Order the result DataFrame by "join_year" and display the result.
df_pin_user_year_age.select("user_name", "age_group", "join_year", "follower_count") \
    .distinct() \
    .filter((col("join_year") >= 2015) & (col("join_year") <= 2020)) \
    .groupBy("join_year") \
    .agg(percentile_approx(col("follower_count"), 0.5).alias("median_follower_count")) \
    .orderBy("join_year") \
    .show()

+---------+---------------------+
join_year|median_follower_count|
+---------+---------------------+
 2015| 92000|
 2016| 21000|
 2017| 6000|
+---------+---------------------+

### Find the median follower count of users based on their joining year and age group

In [ ]:
# Join df_pin and df_user DataFrames on "ind" column to get the required data in df_pin_user.
df_pin_user = df_pin.join(df_user, on="ind")

# Add a new column "join_year" to df_pin_user that contains only the year from the "date_joined" column.
df_pin_user_year = df_pin_user.withColumn("join_year", year("date_joined"))

# Add a new column "age_group" to df_pin_user_year based on the age range of each user.
df_pin_user_year_age = df_pin_user_year.withColumn("age_group", when((col("age") >= 18) & (col("age") <= 24), "18-24")
                                   .when((col("age") >= 25) & (col("age") <= 35), "25-35")
                                   .when((col("age") >= 36) & (col("age") <= 50), "36-50")
                                   .when((col("age") > 50), "50+"))

# Select the columns "user_name", "age_group", "join_year" and "follower_count" from df_pin_user_year_age DataFrame.
# Filter the data for join_years between 2015 and 2020.
# Group the data by "age_group" and "join_year" and calculate the approximate median follower count using percentile_approx() function.
# Rename the aggregated column to "median_follower_count".
# Order the result DataFrame by "age_group" and "join_year" and display the result.
df_pin_user_year_age.select("user_name", "age_group", "join_year", "follower_count") \
    .distinct() \
    .filter((col("join_year") >= 2015) & (col("join_year") <= 2020)) \
    .groupBy("age_group", "join_year") \
    .agg(percentile_approx(col("follower_count"), 0.5).alias("median_follower_count")) \
    .orderBy("age_group", "join_year") \
    .show()

+---------+---------+---------------------+
age_group|join_year|median_follower_count|
+---------+---------+---------------------+
 18-24| 2015| 211000|
 18-24| 2016| 21000|
 18-24| 2017| 940|
 25-35| 2015| 51000|
 25-35| 2016| 43000|
 25-35| 2017| 8000|
 36-50| 2015| 0|
 36-50| 2017| 6000|
 50+| 2016| 0|
+---------+---------+---------------------+